In [1]:
# 처리해야 할 문장을 파이썬 리스트에 옮겨담았습니다.

sentences=['i feel hungry', 'i eat lunch', 'now i feel happy']


# 파이썬 split() 메소드를 이용해 단어 단위로 문장을 쪼개 봅니다.`

word_list = 'i feel hungry'.split()
print(word_list)

['i', 'feel', 'hungry']


In [2]:
index_to_word={}  # 빈 딕셔너리를 만들어서

# 단어들을 하나씩 채워 봅니다. 채우는 순서는 일단 임의로 하였습니다. 그러나 사실 순서는 중요하지 않습니다. 
# <BOS>, <PAD>, <UNK>는 관례적으로 딕셔너리 맨 앞에 넣어줍니다. 
index_to_word[0]='<PAD>'  # 패딩용 단어
index_to_word[1]='<BOS>'  # 문장의 시작지점
index_to_word[2]='<UNK>'  # 사전에 없는(Unknown) 단어
index_to_word[3]='i'
index_to_word[4]='feel'
index_to_word[5]='hungry'
index_to_word[6]='eat'
index_to_word[7]='lunch'
index_to_word[8]='now'
index_to_word[9]='happy'

print(index_to_word)

{0: '<PAD>', 1: '<BOS>', 2: '<UNK>', 3: 'i', 4: 'feel', 5: 'hungry', 6: 'eat', 7: 'lunch', 8: 'now', 9: 'happy'}


In [3]:
word_to_index={word:index for index, word in index_to_word.items()}
print(word_to_index)

{'<PAD>': 0, '<BOS>': 1, '<UNK>': 2, 'i': 3, 'feel': 4, 'hungry': 5, 'eat': 6, 'lunch': 7, 'now': 8, 'happy': 9}


In [4]:
print(word_to_index['feel'])  # 단어 'feel'은 숫자 인덱스 4로 바뀝니다.

4


In [5]:
# 문장 1개를 활용할 딕셔너리와 함께 주면, 단어 인덱스 리스트로 변환해 주는 함수를 만들어 봅시다.
# 단, 모든 문장은 <BOS>로 시작하는 것으로 합니다. 
def get_encoded_sentence(sentence, word_to_index):
    return [word_to_index['<BOS>']]+[word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in sentence.split()]

print(get_encoded_sentence('i eat lunch', word_to_index))

[1, 3, 6, 7]


In [6]:
# 여러 개의 문장 리스트를 한꺼번에 숫자 텐서로 encode해 주는 함수입니다. 
def get_encoded_sentences(sentences, word_to_index):
    return [get_encoded_sentence(sentence, word_to_index) for sentence in sentences]

# sentences=['i feel hungry', 'i eat lunch', 'now i feel happy'] 가 아래와 같이 변환됩니다. 
encoded_sentences = get_encoded_sentences(sentences, word_to_index)
print(encoded_sentences)

[[1, 3, 4, 5], [1, 3, 6, 7], [1, 8, 3, 4, 9]]


In [7]:
# 숫자 벡터로 encode된 문장을 원래대로 decode하는 함수입니다. 
def get_decoded_sentence(encoded_sentence, index_to_word):
    return ' '.join(index_to_word[index] if index in index_to_word else '<UNK>' for index in encoded_sentence[1:])  #[1:]를 통해 <BOS>를 제외

print(get_decoded_sentence([1, 3, 4, 5], index_to_word))

i feel hungry


In [8]:
# 여러개의 숫자 벡터로 encode된 문장을 한꺼번에 원래대로 decode하는 함수입니다. 
def get_decoded_sentences(encoded_sentences, index_to_word):
    return [get_decoded_sentence(encoded_sentence, index_to_word) for encoded_sentence in encoded_sentences]

# encoded_sentences=[[1, 3, 4, 5], [1, 3, 6, 7], [1, 8, 3, 4, 9]] 가 아래와 같이 변환됩니다.
print(get_decoded_sentences(encoded_sentences, index_to_word))

['i feel hungry', 'i eat lunch', 'now i feel happy']


In [13]:
# 아래 코드는 그대로 실행하시면 에러가 발생할 것입니다. 

import numpy as np
import tensorflow as tf
from tensorflow import keras

vocab_size = len(word_to_index)  
# 위 예시에서 딕셔너리에 포함된 단어 개수는 10
word_vector_dim = 4    # 최대 길이가 5이므로 에러나게 됨
# 위 그림과 같이 4차원의 워드벡터를 가정합니다. 
print(vocab_size)
embedding = tf.keras.layers.Embedding(input_dim=vocab_size, 
                                      output_dim=word_vector_dim, mask_zero=True)

# 숫자로 변환된 텍스트 데이터 [[1, 3, 4, 5], [1, 3, 6, 7], [1, 8, 3, 4, 9]] 에 Embedding 레이어를 적용합니다. 
# list 형태의 sentences는 numpy array로 변환되어야 딥러닝 레이어의 입력이 될 수 있습니다.
print(sentences)
print('')

raw_inputs = np.array(get_encoded_sentences(sentences, word_to_index))

raw_inputs = keras.preprocessing.sequence.pad_sequences(raw_inputs,
                                                       value=word_to_index['<PAD>'],
                                                       padding='pre',
                                                       maxlen=5)

print(raw_inputs)
print(type(raw_inputs))
print('')

output = embedding(raw_inputs)
print(output) # 텐서로 변환

10
['i feel hungry', 'i eat lunch', 'now i feel happy']

[[0 1 3 4 5]
 [0 1 3 6 7]
 [1 8 3 4 9]]
<class 'numpy.ndarray'>

tf.Tensor(
[[[ 0.00541806 -0.0035877   0.04206269 -0.04635912]
  [ 0.02585293  0.04718611 -0.0003277   0.03032159]
  [-0.03685191  0.03517352  0.04941038 -0.04787586]
  [-0.0372882   0.03010711  0.01379179  0.00930437]
  [ 0.0094359  -0.02165306  0.00247403 -0.02663261]]

 [[ 0.00541806 -0.0035877   0.04206269 -0.04635912]
  [ 0.02585293  0.04718611 -0.0003277   0.03032159]
  [-0.03685191  0.03517352  0.04941038 -0.04787586]
  [ 0.04760834  0.02651532 -0.02748461 -0.03633936]
  [-0.01754364 -0.00728048  0.02643282  0.04745558]]

 [[ 0.02585293  0.04718611 -0.0003277   0.03032159]
  [-0.02682469  0.04084316  0.01085447  0.0348458 ]
  [-0.03685191  0.03517352  0.04941038 -0.04787586]
  [-0.0372882   0.03010711  0.01379179  0.00930437]
  [ 0.0351564  -0.04048505 -0.03243794  0.04121495]]], shape=(3, 5, 4), dtype=float32)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [10]:
# raw_inputs = keras.preprocessing.sequence.pad_sequences(raw_inputs,
#                                                        value=word_to_index['<PAD>'],
#                                                        padding='pre',
#                                                        maxlen=5)
# print(raw_inputs)

In [11]:
import numpy as np
import tensorflow as tf

vocab_size = len(word_to_index)  # 위 예시에서 딕셔너리에 포함된 단어 개수는 10
word_vector_dim = 3    # 그림과 같이 4차원의 워드벡터를 가정합니다.

embedding = tf.keras.layers.Embedding(input_dim=vocab_size, 
                                      output_dim=word_vector_dim, mask_zero=True)

# keras.preprocessing.sequence.pad_sequences를 통해 word vector를 모두 일정길이로 맞춰주어야 
# embedding 레이어의 input이 될 수 있음에 주의해 주세요. 
raw_inputs = np.array(get_encoded_sentences(sentences, word_to_index))
raw_inputs = keras.preprocessing.sequence.pad_sequences(raw_inputs,
                                                       value=word_to_index['<PAD>'],
                                                       padding='pre',
                                                       maxlen=5)
output = embedding(raw_inputs)
print(output)

tf.Tensor(
[[[ 0.01012838  0.03812069 -0.02426295]
  [-0.04856738 -0.00061896  0.02609006]
  [ 0.0212623  -0.04179021 -0.00641004]
  [ 0.0351553  -0.04685892  0.04944786]
  [ 0.00800328  0.00523306  0.02955787]]

 [[ 0.01012838  0.03812069 -0.02426295]
  [-0.04856738 -0.00061896  0.02609006]
  [ 0.0212623  -0.04179021 -0.00641004]
  [-0.0450337  -0.03786699  0.00936904]
  [ 0.00340655 -0.02875834 -0.03630762]]

 [[-0.04856738 -0.00061896  0.02609006]
  [-0.04248969 -0.04634375 -0.03160607]
  [ 0.0212623  -0.04179021 -0.00641004]
  [ 0.0351553  -0.04685892  0.04944786]
  [-0.03270789  0.02643139  0.00634677]]], shape=(3, 5, 3), dtype=float32)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if sys.path[0] == '':
